In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import pandas as pd
import os

In [2]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
import ast
    
data = pd.read_csv("../../data/DicatJuanRana_w_sentences.csv", sep=";", encoding="utf-8", converters={"sentences": ast.literal_eval})

In [4]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["text", "prev_context", "post_context"],
    template="""
        Actúa como un agente de procesamiento de lenguaje natural cuyo objetivo es aclarar referencias ambiguas en un texto, especialmente aquellas relacionadas con:

         - Sujeto omitido o implícito (ej. "fue", "llegó", "respondió")

         - Complemento directo (ej. "la comedia", "la llamó", "lo rechazaron")

         - Complemento indirecto (ej. "le dijo", "le enviaron")

        Tu tarea es sustituir estas referencias por el nombre o entidad concreta a la que se refieren, usando el contexto de hasta dos frases anteriores y dos posteriores.

        Reescribe únicamente las frases necesarias para que queden claras, manteniendo la coherencia sintáctica. Las frases que ya están completas y claras deben dejarse igual.
        
        Si no puedes determinar a qué se refiere una referencia ambigua, devuelve la frase original sin cambios. No intentes adivinar o hacer suposiciones sobre el significado de la referencia.
        
        No digas que la frase es ambigua, simplemente realiza las sustituciones necesarias.
        
        **EJEMPLO Complemento Directo**

        Entrada:
        La comedia fue muy bien. La comedia era Constantino.

        Salida:
        Constantino fue muy bien. La comedia era Constantino.
        
        **EJEMPLO Sujeto Omitido**

        Entrada:
        Ayer llegaron todos menos Ana. Fue quien organizó el evento.

        Salida:
        Ayer llegaron todos menos Ana. Ana organizó el evento.
        
        Contexto Previo:
        El texto forma parte de la biografía de Cosme Pérez. 
        {prev_context}
        
        Contexto Posterior:
        {post_context}
        
        Texto:
        {text}
    """,
)

In [7]:
from tqdm.auto import tqdm

sentences_of_context = 3 # Number of sentences to consider as context for each sentence
all_clarified_sentences = []

for _, year in tqdm(data.iterrows(), total=len(data)):

    prev_context = []
    post_context = []

    result = []
    texto_to_clarify = year["sentences"]

    for i, sentence in tqdm(enumerate(texto_to_clarify), total=len(texto_to_clarify), leave=False):
        prev_context = result[-2:]
        post_context = texto_to_clarify[i + 1:i + 1 + sentences_of_context]
        
        prev_context = " ".join(prev_context)
        post_context = " ".join(post_context)
        
        response = llm.invoke(
            prompt.format(
                text=sentence,
                prev_context=prev_context,
                post_context=post_context
            )
        ).content
        
        result.append(response)
    
    all_clarified_sentences.append(result)

  0%|          | 0/38 [00:00<?, ?it/s]

  0%|          | 0/93 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
data["clarified_sentences"] = all_clarified_sentences
data.to_csv("../../data/DicatJuanRana_w_clarified_sentences.csv", sep=";", encoding="utf-8", index=False)

In [10]:
import rich as rc

texto_to_clarify = data[data["year"].astype(int) == 1670]["sentences"].values[0]
result = data[data["year"].astype(int) == 1670]["clarified_sentences"].values[0]

print("Texto original:")
rc.print(" ".join(texto_to_clarify))

print("\nTexto corregido:")
rc.print(" ".join(result))

Texto original:


Asistió al cabildo de la Cofradía de Nuestra Señora de la Novena celebrado el 2 de marzo.   El 9 de julio, en 
Madrid, firmó un nuevo testamento que invalidaba el anterior de 1666.   En el que declaraba de nuevo ser vecino de 
Madrid, "que vivo en mis casas propias, en la calle de Cantarranas, Parroquia de San Sebastián, estando en pie 
levantado de la cama, aunque con algunos achaques".   Y pedía ser sepultado "sin ataúd en el convento de religiosos
descalzos de la Santísima Trinidad desta Corte, en la sepultura que allí tengo mía propia, y donde está sepultada 
mi hija Francisca María Pérez".   Aparte del dinero estipulado para misas por su alma y detalles de sus honras, 
entre las limosnas entregaba una de 150 dcs. de vellón para la Cofradía de Nuestra Señora de la Novena para ayuda 
de la fábrica de su capilla.   Y un pabellón de brocado que tenía en el oratorio de su casa.   A su sobrina María 
de Diego y a su sobrino Francisco de Diego, ya mencionados en el testamento anterior, les enviaba 150 dcs. y 30 
dcs. de vellón, respectivamente.   Y 150 dcs. de vellón para ayuda del retablo "que me han dicho se trata de hacer"
para la imagen de Nuestra Señora del Amparo de la iglesia parroquial de Tudela del Duero.   Asimismo, ordenaba 
pagar a su ama Isabel 150 rs. de vellón y lo que se le debiera de salario.   Y a Felipe Negro, su "paisano y 
pariente", otros 150 rs. y un espadín.   Otorgaba a sus albaceas y testamentarios poder para rematar sus bienes y 
pagar todo lo estipulado en el testamento.   Y, con el sobrante, pagar nuevas misas y limosnas para su alma y la de
su hija y familiares.   —A. de la Granja, al publicar el testamento, estimó que el desembolso en las misas 
previstas por su alma debió de rondar los 7.500 rs.   Y destaca la preocupación religiosa que manifiesta Cosme 
Pérez—.


Texto corregido:


Cosme Pérez asistió al cabildo de la Cofradía de Nuestra Señora de la Novena celebrado el 2 de marzo. El 9 de 
julio, en Madrid, Cosme Pérez firmó un nuevo testamento que invalidaba el anterior de 1666. En el que Cosme Pérez 
declaraba de nuevo ser vecino de Madrid, "que vivo en mis casas propias, en la calle de Cantarranas, Parroquia de 
San Sebastián, estando en pie levantado de la cama, aunque con algunos achaques". Y pedía ser sepultado "sin ataúd 
en el convento de religiosos descalzos de la Santísima Trinidad desta Corte, en la sepultura que allí tiene Cosme 
Pérez propia, y donde está sepultada su hija Francisca María Pérez". Aparte del dinero estipulado para misas por el
alma de Cosme Pérez y detalles de sus honras, entre las limosnas entregaba una de 150 dcs. de vellón para la 
Cofradía de Nuestra Señora de la Novena para ayuda de la fábrica de su capilla. Y un pabellón de brocado que Cosme 
Pérez tenía en el oratorio de su casa. A su sobrina María de Diego y a su sobrino Francisco de Diego, ya 
mencionados en el testamento anterior, Cosme Pérez les enviaba 150 dcs. y 30 dcs. de vellón, respectivamente. Y 150
dcs. de vellón para ayuda del retablo "que me han dicho se trata de hacer" para la imagen de Nuestra Señora del 
Amparo de la iglesia parroquial de Tudela del Duero. Asimismo, ordenaba pagar a su ama Isabel 150 rs. de vellón y 
el salario que se le debiera a su ama Isabel. Y a Felipe Negro, su "paisano y pariente", le otorgaba otros 150 rs. 
y un espadín. Otorgaba a sus albaceas y testamentarios poder para rematar los bienes de Cosme Pérez y pagar todo lo
estipulado en el testamento. Y, con el sobrante, Cosme Pérez pagaría nuevas misas y limosnas para su alma y la de 
su hija y familiares. —A. de la Granja, al publicar el testamento de Cosme Pérez, estimó que el desembolso en las 
misas previstas por el alma de Cosme Pérez debió de rondar los 7.500 rs. Y destaca la preocupación religiosa que 
manifiesta Cosme Pérez.

In [29]:
data[data["year"].astype(int) == 1653]["clarified_sentences"].values[0]

['En una carta, fechada en Madrid el 9 de junio de este año, Felipe IV daba cuenta a doña Luisa Enríquez Manrique de la comedia Andrómeda y Perseo, de Calderón, representada en el Coliseo del Buen Retiro en la segunda quincena de mayo:',
 "Muy buena ha sido la comedia Andrómeda y Perseo, y vuestro amigo 'Juan Rana' ha cumplido famosamente con sus obligaciones;",
 'Bien, creo que si hubierais visto a Cosme Pérez, alias Juan Rana, se atrevería la risa a inquietar la función.',
 'La gente moza se ha entretenido harto" —la comedia representada Andrómeda y Perseo, de Calderón, fue Andrómeda y Perseo, en la que Cosme Pérez representó el papel de "Bato"—;',
 'entre la documentación relativa a las fiestas del Corpus de Madrid consta una lista de los gastos de las fiestas del Corpus de este año, en la que se indica que Cosme Pérez recibió 1.000 rs.;',
 'Consta la respuesta, transmitida el 22 de noviembre por el Conde de Altamira, a la solicitud de "Cosme Pérez, alias Juan Rana" realizada el 18 

In [30]:
for _, row in data.iterrows():
    if len(row["clarified_sentences"]) != len(row["sentences"]):
        print(f"Error in year {row['year']}: {len(row['clarified_sentences'])} clarified sentences vs {len(row['sentences'])} original sentences.")
        break

In [11]:
import pandas as pd
import ast
    
data = pd.read_csv("../../data/DicatJuanRana_w_clarified_sentences.csv", sep=";", encoding="utf-8", converters={"sentences": ast.literal_eval, "clarified_sentences": ast.literal_eval})

In [12]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import pandas as pd
import os

In [13]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [14]:
from langchain.prompts import PromptTemplate

comparative_prompt = PromptTemplate(
    input_variables=["original_text", "clarified_text"],
    template="""
        Actúa como un agente de procesamiento de lenguaje natural cuyo objetivo es comparar dos textos y determinar si son iguales o diferentes.

        Tu tarea es analizar el texto original y el texto aclarativo, y responder si son similares o por el contrario, el texto aclarativo muestra información diferente o adicional que no está presente en el texto original.
        
        Responde con "Iguales" si los textos son idénticos en contenido y significado, o devuelve las partes en las que los textos difieren.

        Texto Original:
        {original_text}
        
        Texto Aclarativo:
        {clarified_text}
    """,
)
    

In [15]:
from tqdm.auto import tqdm
import rich as rc

for _, row in tqdm(data.iterrows(), total=len(data)):
    original_text = " ".join(row["sentences"])
    clarified_text = " ".join(row["clarified_sentences"])
    
    response = llm.invoke(
        comparative_prompt.format(
            original_text=original_text,
            clarified_text=clarified_text
        )
    ).content
    print(response)
    # if response != "Iguales":
    #     print(f"{row['year']}")
    #     rc.print(f"Original: {original_text}")
    #     print()
    #     rc.print(f"Clarified: {clarified_text}")
    #     print("\n\n")
        

  0%|          | 0/38 [00:00<?, ?it/s]

Difieren en los siguientes aspectos principales:

1. **Repetición y redundancia en el Texto Aclarativo:**  
   El Texto Aclarativo repite varias veces el nombre "Cosme Pérez" en frases donde en el Texto Original se usa pronombres o formas más naturales, lo que afecta la fluidez pero no cambia el contenido.

2. **Cambios en citas y formulaciones:**  
   - En la cita sobre la popularidad, el Texto Original dice:  
     *"Fue mui zelebrado en la parte de grazioso, y aun excedió a todos los de su tiempo, y solo con salir a las tablas, y sin hablar, probocaba a risa y al aplauso a los que le veían"*.  
     El Texto Aclarativo modifica la cita a:  
     *"Cosme Pérez fue mui zelebrado en la parte de grazioso, y aun excedió a todos los de su tiempo, y solo con salir a las tablas, y sin hablar, probocaba a risa y al aplauso a los que le veían"*.  
     Añade el nombre explícito dentro de la cita, lo que no es fiel al original.

   - Sobre Domingo Canejil, el Texto Original dice:  
     *"porq

KeyboardInterrupt: 